## Import packages

In [1]:
import numpy as np
import numpy_financial as npf
import pandas as pd
import time
import pyomo.environ as pe
from pyomo.gdp import Disjunct, Disjunction
from pyomo.environ import ConstraintList, Block

## Model construction

### Creation of a Concrete Model

In [2]:
model = pe.ConcreteModel()

###  Define sets

#### Products

In [3]:
FEEDSTOCKS = ['hardwood', 'softwood', 'herbaceous-plant']
POLYMERS = ['cellulose','hemicellulose','lignin1']
PRODUCT1 = ['glucose','xylose','furfural','lignin2']
BYPRODUCT = ['glucose','xylose','furfural']
ALL_PRODUCT = FEEDSTOCKS + POLYMERS + PRODUCT1

#### Processes

In [4]:
MILLING = ['milling']
FRACTIONATION = ['DSA','SEP','LHW','organosolv','AFEX','GVL','alkaline']
ALL_PROCESS = MILLING + FRACTIONATION

#### Chemicals and utility

| |  | |
|  :----  | :----  |:----  |
|  ch  | chemicals  |water,NaOH,H2SO4,cellulase-enzymes,EtOH,anthraquinone,H2,CSL,DAP,methanol,chloroform,H3PO4  |
|  ut  | utility  |cooling-water,electricity,natural-gas  |

In [5]:
CHEMICALS = ['water','NaOH','H2SO4','NH3','EtOH','anthraquinone','H2','cellulase-enzymes','CSL','DAP',
             'methanol','chloroform','H3PO4','GVL','glucose','AS','DSP','MPP','boiler_chem','FGD']
UTILITY =['cooling-water','electricity','natural-gas']

### Define parameters

#### Maximum input (ton/day)

In [6]:
max_inflow = 2000

#### Scaling coefficient

In [7]:
scal = 0.6

#### Feedstock composition

In [8]:
composition = pd.DataFrame(index = FEEDSTOCKS)

composition['cellulose'] = [0.46,0.38,0.49]
composition['hemicellulose'] = [0.28,0.32,0.36]
composition['lignin1'] = [0.26,0.3,0.15]

composition

,cellulose,hemicellulose,lignin1
hardwood,0.46,0.28,0.26
softwood,0.38,0.32,0.30
herbaceous-plant,0.49,0.36,0.15


#### Process relationships

In [9]:
process_rel = pd.DataFrame(index = ALL_PROCESS)

process_rel['milling'] = [0,0,0,0,0,0,0,0]
process_rel['DSA'] = [1,0,0,0,0,0,0,0]
process_rel['SEP'] = [1,0,0,0,0,0,0,0]
process_rel['LHW'] = [1,0,0,0,0,0,0,0]
process_rel['organosolv'] = [1,0,0,0,0,0,0,0]
process_rel['AFEX'] = [1,0,0,0,0,0,0,0]
process_rel['GVL'] = [1,0,0,0,0,0,0,0]
process_rel['alkaline'] = [1,0,0,0,0,0,0,0]

process_rel

,milling,DSA,SEP,LHW,organosolv,AFEX,GVL,alkaline
milling,0,1,1,1,1,1,1,1
DSA,0,0,0,0,0,0,0,0
SEP,0,0,0,0,0,0,0,0
LHW,0,0,0,0,0,0,0,0
organosolv,0,0,0,0,0,0,0,0
AFEX,0,0,0,0,0,0,0,0
GVL,0,0,0,0,0,0,0,0
alkaline,0,0,0,0,0,0,0,0


#### Process yield

yield of products from fractionations

|  fractionation | polymer| fractionation product | yield |
|  :----:  | :----:  |:----:  |:----:  |
| DSA | cellulose| glucose | 0.91 |    
| DSA | hemicellulose| xylose | 0.90 |  
| DSA | lignin| lignin | 0.95 |
| DSA | lignin| lignin-monomers | 0 |
| SEP | cellulose| glucose | 0.68 |    
| SEP | hemicellulose| xylose | 0.58 | 
| SEP | lignin| lignin | 0.86 | 
| SEP | lignin| lignin-monomers | 0 | 
| LHW | cellulose| glucose | 1.02 |     
| LHW | hemicellulose| xylose | 0.78 | 
| LHW | lignin| lignin | 0.98 |  
| LHW | lignin| lignin-monomers | 0 |  
| organosolv | cellulose| glucose | 0.9 |    
| organosolv | hemicellulose| xylose | 0.81 |
| organosolv | lignin| lignin | 0.77 |
| organosolv | lignin| lignin-monomers | 0 |
| AFEX | cellulose| glucose | 0.90 |    
| AFEX | hemicellulose| xylose | 0.84 |  
| AFEX | lignin| lignin | 0.95 | 
| AFEX | lignin| lignin-monomers | 0 | 
| GVL | cellulose| glucose | 0.89 |    
| GVL | hemicellulose| furfural | 0.55 |  
| GVL | lignin| lignin | 0.72 | 
| GVL | lignin| lignin-monomers | 0 | 
| alkaline | cellulose| glucose | 0.37 |     
| alkaline | hemicellulose| xylose | 0.17 |  
| alkaline | lignin| lignin | 0 | 
| alkaline | lignin| lignin-monomers | 0.47 |

In [10]:
FRAC_PM = list((a,b) for a in FRACTIONATION for b in POLYMERS)
yield_fr = pd.DataFrame(index = FRAC_PM)

yield_fr['glucose'] = [0.91,0.0,0.0,0.68,0.0,0.0,1,0.0,0.0,0.9,0.0,0.0,0.9,0.0,0.0,0.89,0.0,0.0,0.37,0.0,0.0]
yield_fr['xylose'] = [0.0,0.9,0.0,0.0,0.58,0.0,0.0,0.78,0.0,0.0,0.81,0.0,0.0,0.84,0.0,0.0,0.0,0.0,0.0,0.17,0.0]
yield_fr['furfural'] = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.55,0.0,0.0,0.0,0.0]
yield_fr['lignin2'] = [0.0,0.0,0.95,0.0,0.0,0.86,0.0,0.0,0.98,0.0,0.0,0.77,0,0.0,0.95,0.0,0.0,0.72,0.0,0.0,0.47]

yield_fr

,glucose,xylose,furfural,lignin2
"(DSA, cellulose)",0.91,0.00,0.00,0.00
"(DSA, hemicellulose)",0.00,0.90,0.00,0.00
"(DSA, lignin1)",0.00,0.00,0.00,0.95
"(SEP, cellulose)",0.68,0.00,0.00,0.00
"(SEP, hemicellulose)",0.00,0.58,0.00,0.00
"(SEP, lignin1)",0.00,0.00,0.00,0.86
"(LHW, cellulose)",1.00,0.00,0.00,0.00
"(LHW, hemicellulose)",0.00,0.78,0.00,0.00
"(LHW, lignin1)",0.00,0.00,0.00,0.98
"(organosolv, cellulose)",0.90,0.00,0.00,0.00


#### Process conditions

Chemicals usage of all processes

In [11]:
ch = pd.DataFrame(index = ALL_PROCESS)

ch['water'] = [0,4,4,9,4,4,4,4]
ch['NaOH'] = [0,0,0,0,0,0,0,0.04]
ch['H2SO4'] = [0,0.009,0,0,0.0175,0,0.0392,0.049]
ch['NH3'] = [0,0.003,0,0,0,1,0,0]
ch['EtOH'] = [0,0,0,0,0.032,0,0,0]
ch['anthraquinone'] = [0,0,0,0,0,0,0,0.0005]
ch['H2'] = [0,0,0,0,0,0,0,0]
ch['cellulase-enzymes'] = [0,0.01,0.01,0.01,0.01,0.01,0,0.01]
ch['CSL'] = [0,0,0,0,0,0,0,0]
ch['DAP'] = [0,0,0,0,0,0,0,0]
ch['methanol'] = [0,0,0,0,0,0,0,0]
ch['chloroform'] = [0,0,0,0,0,0,0,0]
ch['H3PO4'] = [0,0,0,0,0,0,0,0]
ch['GVL'] = [0,0,0,0,0,0,2.8,0]
ch['glucose'] = [0,0,0,0,0,0,0,0]
ch['AS'] = [0,0,0,0,0,0,0,0]
ch['DSP'] = [0,0,0,0,0,0,0,0]
ch['MPP'] = [0,0,0,0,0,0,0,0]
ch['boiler_chem'] = [0,0,0,0,0,0,0,0]
ch['FGD'] = [0,0,0,0,0,0,0,0]

ch

,water,NaOH,H2SO4,NH3,EtOH,anthraquinone,H2,cellulase-enzymes,CSL,DAP,methanol,chloroform,H3PO4,GVL,glucose,AS,DSP,MPP,boiler_chem,FGD
milling,0,0.00,0.0000,0.000,0.000,0.0000,0,0.00,0,0,0,0,0,0.0,0,0,0,0,0,0
DSA,4,0.00,0.0090,0.003,0.000,0.0000,0,0.01,0,0,0,0,0,0.0,0,0,0,0,0,0
SEP,4,0.00,0.0000,0.000,0.000,0.0000,0,0.01,0,0,0,0,0,0.0,0,0,0,0,0,0
LHW,9,0.00,0.0000,0.000,0.000,0.0000,0,0.01,0,0,0,0,0,0.0,0,0,0,0,0,0
organosolv,4,0.00,0.0175,0.000,0.032,0.0000,0,0.01,0,0,0,0,0,0.0,0,0,0,0,0,0
AFEX,4,0.00,0.0000,1.000,0.000,0.0000,0,0.01,0,0,0,0,0,0.0,0,0,0,0,0,0
GVL,4,0.00,0.0392,0.000,0.000,0.0000,0,0.00,0,0,0,0,0,2.8,0,0,0,0,0,0
alkaline,4,0.04,0.0490,0.000,0.000,0.0005,0,0.01,0,0,0,0,0,0.0,0,0,0,0,0,0


Utility usage of all processes

In [12]:
ut = pd.DataFrame(index = ALL_PROCESS)

ut['electricity'] = [200,1.19,0.53,1.87,184.13,239.25,175.44,1.18]
ut['cooling-water'] = [0,633.33,819.44,866.67,2.18,1491.67,6922.8,100]
ut['natural-gas'] = [0,544.43,525.05,691.65,595.24,586.08,6433.2,289.16]

ut

,electricity,cooling-water,natural-gas
milling,200.00,0.00,0.00
DSA,1.19,633.33,544.43
SEP,0.53,819.44,525.05
LHW,1.87,866.67,691.65
organosolv,184.13,2.18,595.24
AFEX,239.25,1491.67,586.08
GVL,175.44,6922.80,6433.20
alkaline,1.18,100.00,289.16


#### Product and chemical price

In [13]:
# price of feedstock ($/MT)

price_fs = pd.DataFrame(index = FEEDSTOCKS)
price_fs['price'] = [78.0,87.0,70.0]
price_fs

,price
hardwood,78.0
softwood,87.0
herbaceous-plant,70.0


In [14]:
# price of chemicals ($/MT)

price_ch = pd.DataFrame(index = CHEMICALS)
price_ch['price'] = [3.75,580.4,104.69,462.57,820.46,4643.46,1778.69,1191.97,83.56,387,404.25,590,750,
                     1077.18,893.49,292.02,1039.21,935.29,6601.39,263.57]
price_ch

,price
water,3.75
NaOH,580.40
H2SO4,104.69
NH3,462.57
EtOH,820.46
anthraquinone,4643.46
H2,1778.69
cellulase-enzymes,1191.97
CSL,83.56
DAP,387.00


In [15]:
# price of utility ($/kWh)

price_ut = pd.DataFrame(index = UTILITY)
price_ut['price'] = [0.000795,0.137,0.02]
price_ut

,price
cooling-water,0.000795
electricity,0.137000
natural-gas,0.020000


In [16]:
# price of coproducts ($/MT)

price_by = pd.DataFrame(index = BYPRODUCT)
price_by['price'] = [464.82,464.82,1077.18]
price_by

,price
glucose,464.82
xylose,464.82
furfural,1077.18


#### Process base size and CAPEX

In [17]:
base = pd.DataFrame(index = ALL_PROCESS)

base['base_size'] = [2000,2000,2481,2000,2124,2000,2000,640]
base['base_capex'] = [41.44,145.38,153.34,144.26,136.93,183.4,386.3,58.19]

base

,base_size,base_capex
milling,2000,41.44
DSA,2000,145.38
SEP,2481,153.34
LHW,2000,144.26
organosolv,2124,136.93
AFEX,2000,183.40
GVL,2000,386.30
alkaline,640,58.19


#### Economic factors

In [18]:
n = 20 # operating year
ir = 0.1 # discount rate
tax = 0.21 # tax rate
days = 330 # operating days for a year
work_hour = 7920 # working hours per year
ft = [0.08,0.6,0.32]+[0.0]*27 # fraction of capital expenditure spent at year t

### Define variables

#### Selection of feedstock and processes

In [19]:
# Binary variable for If(fs) and Ip(all process)
model.Ifs = pe.Var(FEEDSTOCKS, domain=pe.Boolean, doc='selection of feedstock')
model.Ip = pe.Var(ALL_PROCESS, domain=pe.Boolean, doc='selection of process')

#### Product flow 

In [20]:
# All processes
model.Fin = pe.Var(ALL_PROCESS, domain=pe.NonNegativeReals, doc='inflow')
model.Fout = pe.Var(ALL_PROCESS, domain=pe.NonNegativeReals, doc='outflow')

# Linkage
model.F = pe.Var(ALL_PROCESS, ALL_PROCESS, ALL_PRODUCT, domain=pe.NonNegativeReals, doc='linkage',initialize=0)

In [21]:
# milling inflow and outflow
model.Finmill = pe.Var(MILLING,POLYMERS,domain=pe.NonNegativeReals,doc='inflow of milling',initialize=0)
model.Foutmill = pe.Var(MILLING,POLYMERS,domain=pe.NonNegativeReals,doc='outflow of milling',initialize=0)

In [22]:
# Fractionation inflow and outflow
model.Finfr = pe.Var(FRACTIONATION,POLYMERS,domain=pe.NonNegativeReals,doc='inflow of fractionation',initialize=0)
model.Foutfr = pe.Var(FRACTIONATION,PRODUCT1,domain=pe.NonNegativeReals,doc='outflow of fractionation',initialize=0)

# Fractionation unconverted part used to burn
model.Fburnfr = pe.Var(FRACTIONATION,POLYMERS,domain=pe.NonNegativeReals,doc='unconverted part of fractionation',initialize=0)

#### Chemicals and utility flow

In [23]:
# chemicals
model.chin = pe.Var(ALL_PROCESS,CHEMICALS,domain=pe.NonNegativeReals,doc='chemical of processes')

# utility
model.utin = pe.Var(ALL_PROCESS,UTILITY,domain=pe.NonNegativeReals,doc='outflow of processes')

### Size fraction

In [24]:
model.sizefraction = pe.Var(ALL_PROCESS,domain = pe.NonNegativeReals,doc = 'size fraction')

### Define constraints

#### Process selection

In [25]:
def one_feedstock_rule(model, i):
    if i == 'hardwood':
        return model.Ifs[i] == 1
    else:
        return model.Ifs[i] == 0
model.onefeedstock = pe.Constraint(FEEDSTOCKS, rule=one_feedstock_rule)

In [26]:
def one_milling_rule(model,i):
    return model.Ip[i] == 1
model.onemilling = pe.Constraint(MILLING,rule=one_milling_rule)

In [27]:
# Select only one fractionation
def one_frac_rule(model, i):
    return sum(model.Ip[i] for i in FRACTIONATION) == 1
model.onefrac = pe.Constraint(FRACTIONATION, rule=one_frac_rule, doc='Select only one fractionation')

In [28]:
# inflow constraint
def inflow_constraint_rule0(model, i):
    return model.Ip[i] <= sum(process_rel[i][j]* model.Ip[j] for j in MILLING)
model.in0constraint = pe.Constraint(FRACTIONATION, rule=inflow_constraint_rule0)

In [29]:
# outflow constraint
def outflow_constraint_rule0(model, i):
    return model.Ip[i] <= sum(process_rel[j][i]* model.Ip[j] for j in FRACTIONATION)
model.out0constraint = pe.Constraint(MILLING, rule=outflow_constraint_rule0)

#### Process product flow

##### Milling

In [30]:
# milling inflow of polymers
def milling_polymer_inflow_rule(model,i,n):
    return model.Finmill[i,n] == sum(model.Ifs[m] * max_inflow * composition[n][m] for m in FEEDSTOCKS)
model.milling_polymer_inflow = pe.Constraint(MILLING, POLYMERS, rule=milling_polymer_inflow_rule, doc='milling polymer inflow')

# milling total inflow 
def milling_inflow_rule(model,i):
    return model.Fin[i] == sum(model.Finmill[i,n] for n in POLYMERS)
model.millinginflow = pe.Constraint(MILLING, rule=milling_inflow_rule, doc='milling total inflow')

In [31]:
# milling outflow of polymers
def milling_polymer_outflow_rule(model,i,n):
    return model.Foutmill[i,n] == model.Finmill[i,n]
model.milling_polymer_outflow = pe.Constraint(MILLING, POLYMERS, rule=milling_polymer_outflow_rule, doc='milling polymer outflow')

# milling outflow
def milling_outflow_rule(model,i):
    return  model.Fout[i] == model.Fin[i]
model.millingoutflow = pe.Constraint(MILLING, rule=milling_outflow_rule, doc='milling total outflow')

In [32]:
# linkage of milling and fractionation
def milling_fractionation_rule(model):
    for i in MILLING:
        for j in FRACTIONATION:
            for n in POLYMERS:
                yield model.F[i,j,n] <= model.Foutmill[i,n] * process_rel[j][i]
                yield model.Foutmill[i,n] == sum(model.F[i,j,n] for j in FRACTIONATION)
model.milling_fractionation = pe.ConstraintList(rule=milling_fractionation_rule)

##### Fractionation

In [33]:
# Fractionation polymer inflow
def frac_polymer_inflow_rule(model,i,n):
    return  model.Finfr[i,n] == sum(model.F[j,i,n] for j in MILLING)
model.frac_polymer_inflow = pe.Constraint(FRACTIONATION, POLYMERS, rule=frac_polymer_inflow_rule, doc='Fractionation polymer inflow')

# Fractionation total inflow
def frac_inflow_rule(model,i):
    return  model.Fin[i] == sum(model.Finfr[i,n] for n in POLYMERS)
model.fracinflow = pe.Constraint(FRACTIONATION, rule=frac_inflow_rule, doc='Fractionation total inflow')

In [34]:
# big M formulation

M = 10*max_inflow

def bigM_fr(model):
    for i in FRACTIONATION:
        for j in MILLING:
            for n in POLYMERS:
                yield model.F[j,i,n] - M * model.Ip[i] <= 0
                yield model.F[j,i,n] >= 0
                
model.bigM_fr = pe.ConstraintList(rule=bigM_fr)

In [35]:
# Fractionation Reaction
def frac_reaction_rule(model,i,n):
    return sum(model.Finfr[i,m] * yield_fr[n][i,m] for m in POLYMERS) == model.Foutfr[i,n]
model.fracreaction = pe.Constraint(FRACTIONATION,PRODUCT1,rule=frac_reaction_rule, doc='Fractionation reaction')

In [36]:
# Unconverted carbohydrates flow to boiler/turbogenerator
def frac_burn_rule(model,i,m):
    return model.Finfr[i,m] * (1 - sum(yield_fr[n][i,m] for n in PRODUCT1)) == model.Fburnfr[i,m]
model.fracunconvert = pe.Constraint(FRACTIONATION,POLYMERS,rule=frac_burn_rule, doc='Fractionation unconverted')

In [37]:
# Fractionation outflow to next stage (lignin)
def frac_outflow_rule(model,i):
    return  model.Fout[i] == model.Foutfr[i,'lignin2'] 
model.fracoutflow = pe.Constraint(FRACTIONATION, rule=frac_outflow_rule, doc='Fractionation outflow to next stage')

#### Size fraction

In [38]:
def size_fraction_rule(model,i):
    return model.sizefraction[i] == model.Fin[i]/base['base_size'][i]
model.fractionrule = pe.Constraint(ALL_PROCESS,rule = size_fraction_rule, doc = 'Size fraction')

#### Chemical and utility flow

In [39]:
# chemical
def chemical_rule(model, i, m):
    if i in FRACTIONATION and m == 'cellulase-enzymes':
        return model.chin[i,m] == model.Finfr[i,'cellulose'] * ch[m][i]
    else:
        return model.chin[i,m] == model.Fin[i] * ch[m][i]
model.chflow = pe.Constraint(ALL_PROCESS, CHEMICALS, rule=chemical_rule, doc='Chemicals flow of process')

In [40]:
# utility
def utility_rule(model, i, m):
    return model.utin[i,m] == model.Fin[i] * ut[m][i]
model.utflow = pe.Constraint(ALL_PROCESS,UTILITY, rule=utility_rule, doc='Utility flow of process')

## Objective

### Boiler/Turbogenerator

In [41]:
# Lower heating value of lignin and carbonhydrates (unit: MWh/MT)

LHV_lignin = 5.6
LHV_carhyd = 4.76

# boiler/turbogenerator power generation efficiency
eff = 0.43 

In [42]:
# Boiler/Turbogenerator inflow (energy)
Fin_BOTU = LHV_lignin * sum(model.Fout[j] * model.Ip[j] for j in FRACTIONATION)\
         + LHV_lignin * sum(model.Fburnfr[j,'lignin1'] for j in FRACTIONATION)\
         + LHV_carhyd * sum(model.Fburnfr[j,'cellulose'] for j in FRACTIONATION)\
         + LHV_carhyd * sum(model.Fburnfr[j,'hemicellulose'] for j in FRACTIONATION)
         
# Boiler/Turbogenerator outflow (power generation, unit: kWh)
Fout_BOTU = eff * Fin_BOTU * 1000

In [43]:
# Equipment cost
base_size_BOTU = 2627
base_EC_BOTU = 90.63

EC_BOTU = base_EC_BOTU * (Fin_BOTU/base_size_BOTU)**scal

In [44]:
# Chemical cost
boiler_chem_BOTU = 1.83e-6
FGD_BOTU = 1.64e-3

ch_BOTU = Fin_BOTU * (boiler_chem_BOTU * price_ch.loc['boiler_chem'].item() + FGD_BOTU * price_ch.loc['FGD'].item())

### Waste Treatment

In [45]:
# equipment cost
base_size_w = 2000
base_EC_w = 41.59

EC_w = base_EC_w*(max_inflow/base_size_w)**scal

In [46]:
# estimated waste treatment operating cost
base_size_w = 2000
base_OPEX_w = 1.69

Cw = base_OPEX_w/base_size_w*max_inflow*1e6/days

### CAPEX

In [47]:
# The calculation is based on NREL and Ke Wang 2021

# Equipment cost
EC = sum((base['base_capex'][i] * (model.sizefraction[i])**scal) for i in ALL_PROCESS) \
   + EC_w + EC_BOTU

In [48]:
# Estimated capital cost
CAPEX = 3.85*EC

In [49]:
# used for result analysis

# Milling Equipment cost
EC0 = sum(model.Ip[i] * (base['base_capex'][i] * (model.sizefraction[i])**scal) for i in MILLING)

# Estimated capital cost
CAPEX0 = 3.85*EC0

In [50]:
# Fractionation Equipment cost
EC1 = sum(model.Ip[i] * (base['base_capex'][i] * (model.sizefraction[i])**scal) for i in FRACTIONATION)

# Estimated capital cost
CAPEX1 = 3.85*EC1

In [51]:
# Boiler/turbogenerator Equipment cost
ECb = EC_BOTU


# Estimated capital cost
CAPEXb = 3.85*ECb

In [52]:
# Waste treatment Equipment cost
ECw = EC_w

# Estimated capital cost
CAPEXw = 3.85*ECw

### OPEX

In [53]:
chemical_cost = sum(model.chin[i,m] * price_ch.loc[m].item() for i in ALL_PROCESS for m in CHEMICALS)\
              + ch_BOTU
utility_cost = sum(model.utin[i,m] * price_ut.loc[m].item() for i in ALL_PROCESS for m in UTILITY)

In [54]:
# fixed operating cost
FOC = CAPEX*1000000 *0.03/days

OPEX= chemical_cost + utility_cost + Cw + FOC

feedstock_cost = sum(model.Ifs[m] * max_inflow * price_fs.loc[m].item() for m in FEEDSTOCKS)

In [55]:
chemical_cost1 = sum(model.Ip[i] * model.chin[i,m] * price_ch.loc[m].item() for i in FRACTIONATION for m in CHEMICALS)
utility_cost1 = sum(model.Ip[i] * model.utin[i,m] * price_ut.loc[m].item() for i in FRACTIONATION for m in UTILITY)

FOC1 = CAPEX1*1000000 *0.03/days

OPEX1= chemical_cost1 + utility_cost1 + FOC1

#### utility usage (result analysis)

In [56]:
# daily utility usage
# UTILITY =['cooling-water','electricity','natural-gas']
electrcity_all = sum(model.Ip[i] * model.utin[i,m] for i in ALL_PROCESS for m in ['electricity'])
electrcity_1 = sum(model.Ip[i] * model.utin[i,m] for i in FRACTIONATION for m in ['electricity'])

heat_all = sum(model.Ip[i] * model.utin[i,m] for i in ALL_PROCESS for m in ['natural-gas'])
heat_1 = sum(model.Ip[i] * model.utin[i,m] for i in FRACTIONATION for m in ['natural-gas'])

water_all = sum(model.Ip[i] * model.utin[i,m] for i in ALL_PROCESS for m in ['cooling-water'])
water_1 = sum(model.Ip[i] * model.utin[i,m] for i in FRACTIONATION for m in ['cooling-water'])

### Annual revenue

In [57]:
# coproducts
revenue_coproduct_glucose = sum(model.Foutfr[i,m] * price_by.loc[m].item() for i in FRACTIONATION for m in ['glucose'])
revenue_coproduct_xylose = sum(model.Foutfr[i,m] * price_by.loc[m].item() for i in FRACTIONATION for m in ['xylose','furfural'])
revenue_coproduct1 = revenue_coproduct_glucose + revenue_coproduct_xylose

In [58]:
# boiler/turbogenerator
revenue_BOTU = Fout_BOTU * price_ut.loc['electricity'].item()

In [59]:
revenue_coproduct = revenue_coproduct1
AR = revenue_coproduct + revenue_BOTU

### Depreciation

In [60]:
DP = [CAPEX/10]*10+[0]*20

### Objective function

In [61]:
EBITDA = (AR - OPEX - feedstock_cost)*days - CAPEX * 1000000/n

In [62]:
CF = [0]*n
NPV = - CAPEX*1000000*ft[0]+DP[0]
CF[0] = NPV

for t in range(1,n):
    CF[t] = - CAPEX*1000000*ft[t]+(AR - OPEX - feedstock_cost)*days*(1-tax) + DP[t]*tax
    NPV += CF[t]/((1+ir)**t)


#NPV = - CAPEX*1000000  + ((AR - OPEX - feedstock_cost)*days) / ir * (1-(1/(1+ir)**n))

In [63]:
model.objective = pe.Objective(expr = NPV, sense = pe.maximize)

In [64]:
# payback time

## Result

In [65]:
model.pprint()

63 Set Declarations
    F_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain                        : Size : Members
        None :     3 : F_index_0*F_index_1*F_index_2 :  640 : {('milling', 'milling', 'hardwood'), ('milling', 'milling', 'softwood'), ('milling', 'milling', 'herbaceous-plant'), ('milling', 'milling', 'cellulose'), ('milling', 'milling', 'hemicellulose'), ('milling', 'milling', 'lignin1'), ('milling', 'milling', 'glucose'), ('milling', 'milling', 'xylose'), ('milling', 'milling', 'furfural'), ('milling', 'milling', 'lignin2'), ('milling', 'DSA', 'hardwood'), ('milling', 'DSA', 'softwood'), ('milling', 'DSA', 'herbaceous-plant'), ('milling', 'DSA', 'cellulose'), ('milling', 'DSA', 'hemicellulose'), ('milling', 'DSA', 'lignin1'), ('milling', 'DSA', 'glucose'), ('milling', 'DSA', 'xylose'), ('milling', 'DSA', 'furfural'), ('milling', 'DSA', 'lignin2'), ('milling', 'SEP', 'hardwood'), ('milling', 'SEP', 'softwood'), ('milling', 'SEP', 'herbaceous-plant')

                           ('LHW', 'water') :     0 :  None :  None : False :  True : NonNegativeReals
                              ('SEP', 'AS') :     0 :  None :  None : False :  True : NonNegativeReals
                             ('SEP', 'CSL') :     0 :  None :  None : False :  True : NonNegativeReals
                             ('SEP', 'DAP') :     0 :  None :  None : False :  True : NonNegativeReals
                             ('SEP', 'DSP') :     0 :  None :  None : False :  True : NonNegativeReals
                            ('SEP', 'EtOH') :     0 :  None :  None : False :  True : NonNegativeReals
                             ('SEP', 'FGD') :     0 :  None :  None : False :  True : NonNegativeReals
                             ('SEP', 'GVL') :     0 :  None :  None : False :  True : NonNegativeReals
                              ('SEP', 'H2') :     0 :  None :  None : False :  True : NonNegativeReals
                           ('SEP', 'H2SO4') :     0 :  None :  None : Fal

        Key  : Active : Sense    : Expression
        None :   True : maximize : - 3.85*(41.44*sizefraction[milling]**0.6 + 145.38*sizefraction[DSA]**0.6 + 153.34*sizefraction[SEP]**0.6 + 144.26*sizefraction[LHW]**0.6 + 136.93*sizefraction[organosolv]**0.6 + 183.4*sizefraction[AFEX]**0.6 + 386.3*sizefraction[GVL]**0.6 + 58.19*sizefraction[alkaline]**0.6 + 41.59 + 90.63*((5.6*(Fout[DSA]*Ip[DSA] + Fout[SEP]*Ip[SEP] + Fout[LHW]*Ip[LHW] + Fout[organosolv]*Ip[organosolv] + Fout[AFEX]*Ip[AFEX] + Fout[GVL]*Ip[GVL] + Fout[alkaline]*Ip[alkaline]) + 5.6*(Fburnfr[DSA,lignin1] + Fburnfr[SEP,lignin1] + Fburnfr[LHW,lignin1] + Fburnfr[organosolv,lignin1] + Fburnfr[AFEX,lignin1] + Fburnfr[GVL,lignin1] + Fburnfr[alkaline,lignin1]) + 4.76*(Fburnfr[DSA,cellulose] + Fburnfr[SEP,cellulose] + Fburnfr[LHW,cellulose] + Fburnfr[organosolv,cellulose] + Fburnfr[AFEX,cellulose] + Fburnfr[GVL,cellulose] + Fburnfr[alkaline,cellulose]) + 4.76*(Fburnfr[DSA,hemicellulose] + Fburnfr[SEP,hemicellulose] + Fburnfr[LHW,hem

In [66]:
solver = pe.SolverFactory('baron')
solver.solve(model, options={'MaxIter': 200,
                            'EpsA':1e-12,
                            'AbsConFeasTol':1e-12,
                            'IsolTol':1e-12}, tee=True)

 BARON version 23.6.23. Built: OSX-64 Fri Jun 23 12:58:03 EDT 2023

 BARON is a product of The Optimization Firm.
 For information on BARON, see https://minlp.com/about-baron
 Licensee: Juliana Vasco-Correa at Penn State University, julianavasco@psu.edu.

 If you use this software, please cite publications from
 https://minlp.com/baron-publications, such as: 

 Khajavirad, A. and N. V. Sahinidis,
 A hybrid LP/NLP paradigm for global optimization relaxations,
 Mathematical Programming Computation, 10, 383-421, 2018.
 This BARON run may utilize the following subsolver(s)
 For LP/MIP/QP: CLP/CBC                                         
 For NLP: IPOPT, FILTERSQP
 Doing local search
 Solving bounding LP
 Preprocessing found feasible solution with value -0.219268E+09
 Starting multi-start local search
 Done with local search
  Iteration    Open nodes         Time (s)    Lower bound      Upper bound
          1             0             0.24    -0.219268E+09    -0.219268E+09   

 Calculating

{'Problem': [{'Name': 'problem', 'Lower bound': -219268311.856, 'Upper bound': -219268092.588, 'Number of objectives': 1, 'Number of constraints': 388, 'Number of variables': 317, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Error rc': 0, 'Time': 0.36803412437438965}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

### solution

In [67]:
def print_solution(model):
    print("Variable Names\t\tValue")
    for c in model.component_data_objects(pe.Var):
        if c.value is None:
            print(c.name,"\t\t UNINITIALIZED")
        else:
            print(c.name,"\t\t", pe.value(c))
        
                
    print("\nObjective Name\t\tValue")
    for c in model.component_data_objects(pe.Objective):
        print(c.name,"\t\t", pe.value(c))

In [68]:
print_solution(model)

Variable Names		Value
Ifs[hardwood] 		 1.0
Ifs[softwood] 		 0.0
Ifs[herbaceous-plant] 		 0.0
Ip[milling] 		 1.0
Ip[DSA] 		 1.0
Ip[SEP] 		 0.0
Ip[LHW] 		 0.0
Ip[organosolv] 		 0.0
Ip[AFEX] 		 0.0
Ip[GVL] 		 0.0
Ip[alkaline] 		 0.0
Fin[milling] 		 2000.0
Fin[DSA] 		 2000.0
Fin[SEP] 		 0.0
Fin[LHW] 		 0.0
Fin[organosolv] 		 0.0
Fin[AFEX] 		 0.0
Fin[GVL] 		 0.0
Fin[alkaline] 		 0.0
Fout[milling] 		 2000.0
Fout[DSA] 		 493.99999999999994
Fout[SEP] 		 0.0
Fout[LHW] 		 0.0
Fout[organosolv] 		 0.0
Fout[AFEX] 		 0.0
Fout[GVL] 		 0.0
Fout[alkaline] 		 0.0
F[milling,milling,hardwood] 		 0
F[milling,milling,softwood] 		 0
F[milling,milling,herbaceous-plant] 		 0
F[milling,milling,cellulose] 		 0
F[milling,milling,hemicellulose] 		 0
F[milling,milling,lignin1] 		 0
F[milling,milling,glucose] 		 0
F[milling,milling,xylose] 		 0
F[milling,milling,furfural] 		 0
F[milling,milling,lignin2] 		 0
F[milling,DSA,hardwood] 		 0
F[milling,DSA,softwood] 		 0
F[milling,DSA,herbaceous-plant] 		 0
F[milling,DSA,

F[LHW,organosolv,softwood] 		 0
F[LHW,organosolv,herbaceous-plant] 		 0
F[LHW,organosolv,cellulose] 		 0
F[LHW,organosolv,hemicellulose] 		 0
F[LHW,organosolv,lignin1] 		 0
F[LHW,organosolv,glucose] 		 0
F[LHW,organosolv,xylose] 		 0
F[LHW,organosolv,furfural] 		 0
F[LHW,organosolv,lignin2] 		 0
F[LHW,AFEX,hardwood] 		 0
F[LHW,AFEX,softwood] 		 0
F[LHW,AFEX,herbaceous-plant] 		 0
F[LHW,AFEX,cellulose] 		 0
F[LHW,AFEX,hemicellulose] 		 0
F[LHW,AFEX,lignin1] 		 0
F[LHW,AFEX,glucose] 		 0
F[LHW,AFEX,xylose] 		 0
F[LHW,AFEX,furfural] 		 0
F[LHW,AFEX,lignin2] 		 0
F[LHW,GVL,hardwood] 		 0
F[LHW,GVL,softwood] 		 0
F[LHW,GVL,herbaceous-plant] 		 0
F[LHW,GVL,cellulose] 		 0
F[LHW,GVL,hemicellulose] 		 0
F[LHW,GVL,lignin1] 		 0
F[LHW,GVL,glucose] 		 0
F[LHW,GVL,xylose] 		 0
F[LHW,GVL,furfural] 		 0
F[LHW,GVL,lignin2] 		 0
F[LHW,alkaline,hardwood] 		 0
F[LHW,alkaline,softwood] 		 0
F[LHW,alkaline,herbaceous-plant] 		 0
F[LHW,alkaline,cellulose] 		 0
F[LHW,alkaline,hemicellulose] 		 0
F[LHW,alkal

F[alkaline,milling,hemicellulose] 		 0
F[alkaline,milling,lignin1] 		 0
F[alkaline,milling,glucose] 		 0
F[alkaline,milling,xylose] 		 0
F[alkaline,milling,furfural] 		 0
F[alkaline,milling,lignin2] 		 0
F[alkaline,DSA,hardwood] 		 0
F[alkaline,DSA,softwood] 		 0
F[alkaline,DSA,herbaceous-plant] 		 0
F[alkaline,DSA,cellulose] 		 0
F[alkaline,DSA,hemicellulose] 		 0
F[alkaline,DSA,lignin1] 		 0
F[alkaline,DSA,glucose] 		 0
F[alkaline,DSA,xylose] 		 0
F[alkaline,DSA,furfural] 		 0
F[alkaline,DSA,lignin2] 		 0
F[alkaline,SEP,hardwood] 		 0
F[alkaline,SEP,softwood] 		 0
F[alkaline,SEP,herbaceous-plant] 		 0
F[alkaline,SEP,cellulose] 		 0
F[alkaline,SEP,hemicellulose] 		 0
F[alkaline,SEP,lignin1] 		 0
F[alkaline,SEP,glucose] 		 0
F[alkaline,SEP,xylose] 		 0
F[alkaline,SEP,furfural] 		 0
F[alkaline,SEP,lignin2] 		 0
F[alkaline,LHW,hardwood] 		 0
F[alkaline,LHW,softwood] 		 0
F[alkaline,LHW,herbaceous-plant] 		 0
F[alkaline,LHW,cellulose] 		 0
F[alkaline,LHW,hemicellulose] 		 0
F[alkaline,LHW

In [69]:
print("net present value",f'{NPV():,}')

print("EBITDA",f'{EBITDA():,}')

print("annual cash flow",f'{AR()*days-OPEX()*days-feedstock_cost()*days:,}')

print("capital cost",f'{CAPEX()*1000000:,}')
print("capital cost (annulized with interest rate)",f'{CAPEX()*1000000* ir / (1-(1/(1+ir)**n)):,}')
print("annual operating cost",f'{OPEX()*days:,}')
print("annual chemicals cost",f'{chemical_cost()*days:,}')
print("annual utility cost",f'{utility_cost()*days:,}')
print("annual wastewater treatment cost",f'{Cw*days:,}')
print("annual feedstock cost",f'{feedstock_cost()*days:,}')
print("annual fixed operating cost",f'{FOC()*days:,}')
print("total annual cost",f'{CAPEX()*1000000* ir / (1-(1/(1+ir)**n))+OPEX()*days+feedstock_cost()*days:,}')
print("annual fractionation cost",f'{CAPEX1()*1000000* ir / (1-(1/(1+ir)**n))+OPEX1()*days:,}')

print("total annual revenue",f'{AR()*days:,}')
print("annual revenue from fractionation",f'{revenue_coproduct1()*days:,}')

print("annual revenuet from glucose",f'{revenue_coproduct_glucose()*days:,}')
print("annual revenue from xylose",f'{revenue_coproduct_xylose()*days:,}')   
print("annual revenue from coproduct",f'{revenue_coproduct()*days:,}') 
print("annual revenue from boiler/turbogenerator",f'{revenue_BOTU()*days:,}') 

net present value -219,268,311.85624307
EBITDA 76,801,091.52403544
annual cash flow 141,750,897.7363656
capital cost 1,298,996,124.2466035
capital cost (annulized with interest rate) 152,579,597.33499727
annual operating cost 81,950,701.51003435
annual chemicals cost 15,580,433.731636245
annual utility cost 25,710,384.051000003
annual wastewater treatment cost 1,690,000.0
annual feedstock cost 51,480,000.0
annual fixed operating cost 38,969,883.727398105
total annual cost 286,010,298.8450316
annual fractionation cost 105,218,021.1313159
total annual revenue 275,181,599.24639994
annual revenue from fractionation 205,727,472.71999997
annual revenuet from glucose 128,418,610.31999998
annual revenue from xylose 77,308,862.39999999
annual revenue from coproduct 205,727,472.71999997
annual revenue from boiler/turbogenerator 69,454,126.52639998


In [70]:
print("annual electricity from boiler/turbogenerator",f'{Fout_BOTU()*days:,}') 

annual electricity from boiler/turbogenerator 506,964,427.19999987


In [71]:
cash_flow = []
for i in range(n):
    cash_flow.append(CF[i]())
    
# Calculate the IRR
irr = npf.irr(cash_flow)

# Print the IRR as a percentage
print(f"The IRR is: {irr * 100:.2f}%")

The IRR is: 6.50%


### Utility usage

In [72]:
# annual utility usage
# UTILITY =['cooling-water','electricity','natural-gas']
electrcity_all = days*sum(model.Ip[i] * model.utin[i,m] for i in ALL_PROCESS for m in ['electricity'])
electrcity_milling = days* sum(model.Ip[i] * model.utin[i,m] for i in MILLING for m in ['electricity'])
electrcity_1 = days* sum(model.Ip[i] * model.utin[i,m] for i in FRACTIONATION for m in ['electricity'])

heat_all = days*sum(model.Ip[i] * model.utin[i,m] for i in ALL_PROCESS for m in ['natural-gas'])
heat_milling = days*sum(model.Ip[i] * model.utin[i,m] for i in MILLING for m in ['natural-gas'])
heat_1 = days*sum(model.Ip[i] * model.utin[i,m] for i in FRACTIONATION for m in ['natural-gas'])

water_all = days*sum(model.Ip[i] * model.utin[i,m] for i in ALL_PROCESS for m in ['cooling-water'])
water_milling = days*sum(model.Ip[i] * model.utin[i,m] for i in MILLING for m in ['cooling-water'])
water_1 = days*sum(model.Ip[i] * model.utin[i,m] for i in FRACTIONATION for m in ['cooling-water'])

In [73]:
print(f'annual electrcity usage is {electrcity_all()/1000:,} mwh')
print(f'annual electrcity usage in milling is {electrcity_milling()/1000:,} mwh')
print(f'annual electrcity usage in fractionation is {electrcity_1()/1000:,} mwh')

annual electrcity usage is 132,785.4 mwh
annual electrcity usage in milling is 132,000.0 mwh
annual electrcity usage in fractionation is 785.4 mwh


In [74]:
print(f'annual natural gas usage is {heat_all()/1000:,} mwh')
print(f'annual natural gas usage in milling is {heat_milling()/1000:,} mwh')
print(f'annual natural gas usage in fractionation is {heat_1()/1000:,} mwh')

annual natural gas usage is 359,323.8 mwh
annual natural gas usage in milling is 0.0 mwh
annual natural gas usage in fractionation is 359,323.8 mwh


In [75]:
print(f'annual cooling water usage is {water_all()/1000:,} mwh')
print(f'annual cooling water usage in milling is {water_milling()/1000:,} mwh')
print(f'annual cooling water usage in fractionation is {water_1()/1000:,} mwh')

annual cooling water usage is 417,997.8 mwh
annual cooling water usage in milling is 0.0 mwh
annual cooling water usage in fractionation is 417,997.8 mwh
